<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250722_NLP_PDF_%EC%9A%94%EC%95%BD%ED%85%8C%EC%8A%A4%ED%8A%B8_3_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pdfplumber & kobart-summarization 모델 사용

## 1. 패키지 설치

In [ ]:
# ✅ STEP 1. 설치
!pip install pdfplumber transformers

In [ ]:
!pip install pdfplumber transformers sentencepiece


##  2. Google Drive 마운트 & 모델 로드

In [ ]:
import pdfplumber, re, torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from google.colab import drive

# 드라이브 마운트
drive.mount('/content/drive')

# KoBART Summarization 모델 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization').to('cuda' if torch.cuda.is_available() else 'cpu')


# 3. PDF 텍스트 추출

In [ ]:
# PDF 경로 설정
pdf_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/강원랜드 사업보고서 (2024.12).pdf"

raw_texts = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            raw_texts.append(text)

full_text = "\n".join(raw_texts)
clean_text = re.sub(r'\s+', ' ', full_text)
print("✅ 전체 추출 텍스트 길이:", len(clean_text))


# 4. 슬라이딩 윈도우 요약

In [ ]:
import pandas as pd
import torch

# 슬라이딩 윈도우 파라미터
window_size = 2800
step_size = 2500

summaries = []
summary_list = []

print("📚 슬라이딩 요약 시작...\n")

for i in range(0, len(clean_text), step_size):
    chunk = clean_text[i:i+window_size]
    chunk_preview = chunk[:200].strip()

    print(f"\n🔹 [{i//step_size + 1}번째 chunk]")
    print(f"▶ 입력 일부:\n{chunk_preview}...\n")

    # ✅ attention_mask, padding 포함한 인코딩
    inputs = tokenizer(chunk, return_tensors='pt', max_length=1024, truncation=True, padding='max_length')
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    # ✅ 오류 방지: pad_token_id 명시 + max_new_tokens 사용
    with torch.no_grad():
        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=200,  # 기존 max_length → max_new_tokens
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id  # 경고 방지
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

    print(f"✅ 요약 결과:\n{summary}\n{'='*50}")


    # # CSV 저장 여부 확인
    # to_csv = input("CSV로 저장하시겠습니까? (y/n): ")
    # if to_csv.lower() == 'y':
    #     summary_list.append({
    #         'index': i + 1,
    #         'summary': summary
    #     })

    # # 전체 저장
    # if summary_list:
    #     df = pd.DataFrame(summary_list)
    #     df.to_csv("요약결과.csv", index=False, encoding='utf-8-sig')
    #     print("✅ 요약 결과가 CSV 파일로 저장되었습니다.")
    # else:
    #     print("❌ 저장할 요약이 없습니다.")


## 저장 코드

In [ ]:
# 저장 코드
import pickle

# 요약 결과 저장
with open("summaries.pkl", "wb") as f:
    pickle.dump(summaries, f)

print("✅ 현재 summaries 리스트가 저장되었습니다.")

from google.colab import files
files.download("summaries.pkl")



# 5. 전체 종합 요약 출력

## 불러오기 코드

In [57]:
import pickle
import os

# 저장된 피클 파일 경로
save_path = '/content/drive/MyDrive/Colab Notebooks/실전프로젝트/summaries.pkl'

# 이미 저장된 데이터 로드
with open(save_path, "rb") as f:
    collected_data = pickle.load(f)

print(f"✅ 저장된 데이터 수: {len(collected_data)}개")

✅ 저장된 데이터 수: 105개


In [58]:
print(f"🔄 저장된 데이터 개수: {len(collected_data)}")
print(f"📌 마지막 처리 기업 예시:\n{collected_data[-1]}")

🔄 저장된 데이터 개수: 105
📌 마지막 처리 기업 예시:
의 예측가능성 및 안정성을 강화하기 위해 3년 단위 주주환원정책을제시하 고 있습니다. 2024년에 발표된 기업가치 제고계획에 따른 2024~2026회계연도주주환원정 책은 다음과 같습니다. 1) 총주주환원율 60% 달성 * 총주주환원율 = 연결재무제표 기준 당기순이익 / 총주주환원액 * 100 * 총주주환원액 = 배당액 + 자기주식 매입액 2) 배당성향 최소 50% 이상 * 배당성향 = 연결재무제표 기준 당기순이익 / 배당액 * 100 3) 자기주식 매입 총 1,000억원(2024년 400억원, 2025년~2026년 600억원) * 2024년 400억원(매입 완료) * 2025년 및 2026년 매입금액 미정(보고서 제출일 현재) 나. 배당관련 예측가능성 제공에 관한 사항 1) 정관상 배당절차 개선방안 이행 가부 구분 현황 및 계획 정관상 배당액 결정 기관 주주총회 가능 (정관 제41조 제2항에 근거) 정관상 배당기준일을 배당액 결정 이후로 정할 ※ 당사는 2024회계연도 결산배당부터 수 있는지 여부 배당기준일을 배당액 확정일(정기주주총회일) 이후의 날로 정하기로 함 배당절차 개선방안 이행 관련 향후 계획 - 2) 배당액 확정일 및 배당기준일 지정 현황 배당액 배당 예측가능성 구분 결산월 배당여부 배당기준일 비고 확정일 제공여부 배당기준일, 배당(안) 2024회계연도 2024.12 O 2025.03.26 2025.04.04 O 결정 및 공시일 : 결산배당 2025.3.7 2023회계연도 배당(안) 결정 및 공시일 : 2023.12 O 2024.03.28 2023.12.31 X 결산배당 2024.3.8 전자공시시스템 dart.fss.or.kr Page 204 2022회계연도 배당(안) 결정 및 공시일 : 2022.12 O 2023.03.29 2022.12.31 X 결산배당 2023.3.9 ※ 배당액 확정일 : 정기주주총회일 다. 주요 배당지표 당기 전기 전전기 구 분 주식의 종류 제27기 제26기 제25기

In [59]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/실전프로젝트/summaries.pkl', "rb") as f:
  summaries = pickle.load(f)


print(f"✅ 불러온 summaries 개수: {len(summaries)}개")


✅ 불러온 summaries 개수: 105개


In [60]:
final_summary = "\n\n".join([f"[{i+1}] {s}" for i, s in enumerate(summaries)])

In [61]:
print("📘 전체 종합 요약 (일부 미리보기):\n")
print(final_summary[150:50000] + "\n... [생략]")


📘 전체 종합 요약 (일부 미리보기):

.......1 【 대표이사 등의 확인 】...........................................................................................................................2 I. 회사의 개요............................................................................................................................................3 1. 회사의 개요......................................................................................................................................3 2. 회사의 연혁......................................................................................................................................4 3. 자본금 변동사항...............................................................................................................................6 4. 주식의 총수 등.................................................................................................................................6 5. 정관에 관한 사항..............................................................................................

In [62]:
with open("요약결과_전체.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)


In [63]:
from google.colab import files
files.download("요약결과_전체.txt")

print("📄 전체 요약이 txt 파일로 저장되었습니다.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📄 전체 요약이 txt 파일로 저장되었습니다.


In [64]:
start_index = len(collected_data)  # 다음부터 처리할 기업 인덱스


In [65]:
# 불러온 summaries 리스트를 활용해서 전체 요약 출력하거나 저장하는 흐름

# 1. 전체 요약 병합
final_summary = "\n\n".join([f"[{i+1}] {s}" for i, s in enumerate(summaries)])

# 2. 일부 출력
print("📘 전체 종합 요약 (일부 미리보기):\n")
print(final_summary[150:50000] + "\n... [생략]")

# 3. 파일로 저장
with open("요약결과_전체.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

print("✅ 요약이 텍스트 파일로 저장되었습니다.")


📘 전체 종합 요약 (일부 미리보기):

.......1 【 대표이사 등의 확인 】...........................................................................................................................2 I. 회사의 개요............................................................................................................................................3 1. 회사의 개요......................................................................................................................................3 2. 회사의 연혁......................................................................................................................................4 3. 자본금 변동사항...............................................................................................................................6 4. 주식의 총수 등.................................................................................................................................6 5. 정관에 관한 사항..............................................................................................

In [66]:
!pip install pdfplumber

In [67]:
import pdfplumber

pdf_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/강원랜드 사업보고서 (2024.12).pdf"



# ✅ PDF 경로 설정 (원하는 파일로 수정)
pdf_path

# ✅ 텍스트 추출
with pdfplumber.open(pdf_path) as pdf:
    text = ''.join(page.extract_text() for page in pdf.pages)

# ✅ 전처리
clean_text = text.replace('\n', ' ').replace('�', '').strip()

print(f"✅ 텍스트 추출 완료: 총 {len(clean_text)}자")


✅ 텍스트 추출 완료: 총 326659자


In [68]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# ✅ 모델명: 한국어 요약 특화 KoBART (digit82 버전)
model_name = "digit82/kobart-summarization"

# ✅ tokenizer 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

# ✅ 모델 불러오기
model = BartForConditionalGeneration.from_pretrained(model_name)

# ✅ device 설정 (GPU가 있다면 CUDA 사용)
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("✅ KoBART 모델과 토크나이저 로딩 완료")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


✅ KoBART 모델과 토크나이저 로딩 완료


In [69]:
import torch

# ✅ 슬라이딩 윈도우 파라미터
window_size = 2800
step_size = 2500

# ✅ 이어서 시작할 인덱스 계산
start_chunk = len(summaries)
start_pos = start_chunk * step_size

print("📚 슬라이딩 요약 시작...\n")

for i in range(start_pos, len(clean_text), step_size):
    chunk = clean_text[i:i+window_size]
    chunk_preview = chunk[:200].strip()

    print(f"\n🔹 [{i//step_size + 1}번째 chunk]")
    print(f"▶ 입력 일부:\n{chunk_preview}...\n")

    # ✅ tokenizer와 model은 사전 로딩되어 있어야 함
    inputs = tokenizer(chunk, return_tensors='pt', max_length=1024, truncation=True, padding='max_length')
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    with torch.no_grad():
        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=200,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

    print(f"✅ 요약 결과:\n{summary}\n{'='*50}")


📚 슬라이딩 요약 시작...


🔹 [106번째 chunk]
▶ 입력 일부:
1년초과 2년초과 3년 초과 합 계 전자공시시스템 dart.fss.or.kr Page 20630일이하 90일이하 180일이하 1년이하 2년이하 3년이하 공모 - - - - - - - - - 미상환 사모 - - - - - - - - - 잔액 합계 - - - - - - - - - 단기사채 미상환 잔액 (기준일 : - ) (단위 : 원) 10일초과 30일초과 9...

✅ 요약 결과:
1년초과 2년초과 3년 초과 합 계 전자공시시스템 - - - - - - - - - - -

🔹 [107번째 chunk]
▶ 입력 일부:
서 사용되는 식음료 자재 및 판매용 상품과 머신제조 제품 으로 구성되어 있으며 재고자산의 보유현황은 다음과 같습니다. (단위 : 원) 계정과목 제27기 제26기 제25기 제 품 640,178,126 1,277,256,951 1,238,650,281 상 품 931,254,041 733,884,922 626,233,811 식음료 및 머신원자재 2,831,939...

✅ 요약 결과:
제 품, 제26기 제25기 제 품 640,178,126 1,277,256,256,951 1,238,650,650,281 상 품 931,254,041 733,884,041 733,884,922 626,233,811 식음료 및 머신원자재 및 머신원자재 2,831,939,801 합 계 4,403,373,371,973 5,522,131,094 4,094 4,094 4,094 4,095,893 총자산대비 재고자산 구성비율(%) 0.09 0.12 0.10 [재고자산÷기말자산총계×100] 재고자산회전율(횟수)_리조트및머신매출원가 기준 48회 51회 62회

🔹 [108번째 chunk]
▶ 입력 일부:
측정치는 다음과 같습니다. <당분기말> (단위: 원) 구 분 장부금액 수준1 수준2 수준3 당기손익-공정가치 측정 금융자산 집합투자증권 등 2,041,700,238,432 991,915,684,362 1,049,78

In [70]:
# 리스트 병합
final_summary = "\n\n".join([f"[{i+1}] {s}" for i, s in enumerate(summaries)])

# 일부 미리보기 출력
print("📘 전체 요약 (앞부분 일부):\n")
print(final_summary[:3000] + "\n... [이하 생략]")


📘 전체 요약 (앞부분 일부):

[1] 목 차 사 업 보 고 서............................................................................................................................................1 【 대표이사 등의 확인 】...........................................................................................................................2 I. 회사의 개요............................................................................................................................................3 1. 회사의 개요......................................................................................................................................3 2. 회사의 연혁......................................................................................................................................4 3. 자본금 변동사항...............................................................................................................................6 4. 주식의 총수 등...........................................................................................

In [71]:
with open("요약결과_전체.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

print("✅ 최종 요약 텍스트가 저장되었습니다.")

✅ 최종 요약 텍스트가 저장되었습니다.


In [72]:
!ls /content/요약결과_전체.txt

/content/요약결과_전체.txt


In [73]:
with open("요약결과_전체.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

In [74]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. 텍스트 파일 로드
with open("/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_요약결과전체.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("📥 텍스트 파일 로드 완료")

# 2. [청크 번호] 제거 및 줄바꿈 정리
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\n+', ' ', text)

# 3. 문장 분할
sentences = re.split(r'(?<=[.?!])\s+', text)
print(f"✂️ 총 {len(sentences)}개의 문장으로 분리됨")

# 4. 중복 문장 제거 (TF-IDF + cosine similarity)
def remove_duplicates_fast(sentences, threshold=0.8):
    tfidf = TfidfVectorizer().fit_transform(sentences)
    sim_matrix = cosine_similarity(tfidf)
    keep = []
    for i in range(len(sentences)):
        if all(sim_matrix[i][j] < threshold for j in keep):
            keep.append(i)
    return [sentences[i] for i in keep]

clean_sentences = remove_duplicates_fast(sentences)
print(f"🧹 중복 제거 후 문장 수: {len(clean_sentences)}")

# 5. 문장 키워드 기반 항목 분류
sections = {
    "📊 재무 요약": [],
    "🛡️ 리스크 및 부채": [],
    "🚀 사업 전략 및 성장": [],
    "🔬 기술 및 R&D": [],
    "🌐 글로벌 진출": [],
    "🏢 조직 및 인력": [],
    "기타": []
}

keywords = {
    "📊 재무 요약": ["매출", "이익", "적자", "흑자", "수익", "순이익", "영업"],
    "🛡️ 리스크 및 부채": ["부채", "위험", "리스크", "안정성", "재무 위험", "지급"],
    "🚀 사업 전략 및 성장": ["전략", "시장", "성장", "확대", "계획", "기회", "투자"],
    "🔬 기술 및 R&D": ["기술", "R&D", "연구", "개발", "특허", "혁신"],
    "🌐 글로벌 진출": ["해외", "글로벌", "수출", "진출", "국가"],
    "🏢 조직 및 인력": ["조직", "인력", "직원", "채용", "구성", "팀"]
}

def classify_sentence(s):
    for section, kws in keywords.items():
        if any(kw in s for kw in kws):
            return section
    return "기타"

for s in clean_sentences:
    section = classify_sentence(s)
    sections[section].append(s)

print("📚 문장 항목별 분류 완료")

# 6. 문단 제목 + 문단당 최대 5문장으로 구성
final_output = ""
for title, sents in sections.items():
    if not sents:
        continue
    final_output += f"\n\n## {title}\n"
    for i in range(0, len(sents), 5):
        paragraph = " ".join(sents[i:i+5])
        final_output += paragraph.strip() + "\n\n"

# 7. 저장
with open("/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_정제_분류형.txt", "w", encoding="utf-8") as f:
    f.write(final_output.strip())

print("✅ 정제 및 분류된 요약 결과 저장 완료")


📥 텍스트 파일 로드 완료
✂️ 총 1442개의 문장으로 분리됨
🧹 중복 제거 후 문장 수: 1196
📚 문장 항목별 분류 완료
✅ 정제 및 분류된 요약 결과 저장 완료


In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 텍스트 파일 로드
with open("/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_요약결과전체.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("📥 텍스트 파일 로드 완료")

# 2. 청크 번호 제거 및 줄바꿈 정리
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\n+', ' ', text)

# 3. 문장 분할
sentences = re.split(r'(?<=[.?!])\s+', text)
print(f"✂️ 총 {len(sentences)}개의 문장으로 분리됨")

# 4. 중복 문장 제거 (TF-IDF + cosine similarity)
def remove_duplicates_fast(sentences, threshold=0.8):
    tfidf = TfidfVectorizer().fit_transform(sentences)
    sim_matrix = cosine_similarity(tfidf)
    keep = []
    for i in range(len(sentences)):
        if all(sim_matrix[i][j] < threshold for j in keep):
            keep.append(i)
    return [sentences[i] for i in keep]

clean_sentences = remove_duplicates_fast(sentences)
print(f"🧹 중복 제거 후 문장 수: {len(clean_sentences)}")

# 5. 문장 키워드 기반 항목 분류
sections = {
    "📊 재무 요약": [],
    "🛡️ 리스크 및 부채": [],
    "🚀 사업 전략 및 성장": [],
    "🔬 기술 및 R&D": [],
    "🌐 글로벌 진출": [],
    "🏢 조직 및 인력": [],
    "기타": []
}

keywords = {
    "📊 재무 요약": ["매출", "이익", "적자", "흑자", "수익", "순이익", "영업"],
    "🛡️ 리스크 및 부채": ["부채", "위험", "리스크", "안정성", "재무 위험", "지급"],
    "🚀 사업 전략 및 성장": ["전략", "시장", "성장", "확대", "계획", "기회", "투자"],
    "🔬 기술 및 R&D": ["기술", "R&D", "연구", "개발", "특허", "혁신"],
    "🌐 글로벌 진출": ["해외", "글로벌", "수출", "진출", "국가"],
    "🏢 조직 및 인력": ["조직", "인력", "직원", "채용", "구성", "팀"]
}

def classify_sentence(s):
    for section, kws in keywords.items():
        if any(kw in s for kw in kws):
            return section
    return "기타"

for s in clean_sentences:
    section = classify_sentence(s)
    sections[section].append(s)

print("📚 문장 항목별 분류 완료")

# 6. 항목별 최대 1문장 선별 (TF-IDF 상위)
def select_top_sentences(sentences, top_k=1):
    if len(sentences) <= top_k:
        return sentences
    tfidf = TfidfVectorizer().fit_transform(sentences)
    scores = np.asarray(tfidf.sum(axis=1)).ravel()  # 문장별 tf-idf 총합
    top_idx = np.argsort(scores)[-top_k:][::-1]     # 상위 인덱스
    return [sentences[i] for i in sorted(top_idx)]

# 7. 출력 구성
final_output = ""
for title, sents in sections.items():
    if not sents:
        continue
    selected = select_top_sentences(sents, top_k=3)
    final_output += f"\n\n## {title}\n"
    for s in selected:
        final_output += s.strip() + "\n"
    final_output += "\n"

# 8. 저장
with open("/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_정제 요약본 1문장.txt", "w", encoding="utf-8") as f:
    f.write(final_output.strip())

print("✅ 최종 요약 파일 저장 완료 (항목당 최대 1문장)")


📥 텍스트 파일 로드 완료
✂️ 총 21개의 문장으로 분리됨
🧹 중복 제거 후 문장 수: 21
📚 문장 항목별 분류 완료
✅ 최종 요약 파일 저장 완료 (항목당 최대 1문장)


In [5]:
print(final_output)



## 📊 재무 요약
## 📊 재무 요약 요약 연결 재무정보(K-IFRS) (단위 : 원) 제 27 기 제 26 기 제 25 기 구 분 (2024년 12월말) (2023년 12월말) (2022년 12월말) [유동자산] 1,213,770,948,090 1,391,068,809,421 1,328,836,088,014 ㆍ현금및현금성자산 244,499,636,973 166,186,546,142 101,060,567,371 ㆍ유동금융자산 932,676,085,313 1,187,637,044,467 1,198,553,807,206 ㆍ매출채권및기타채권 25,716,727,890 24,172,481,246 22,394,067,535 ㆍ재고자산 4,403,371,973 5,522,131,094 4,027,395,893 ㆍ당기법인세자산 - 3,560,918,544 1,136,530 ㆍ유동비금융자산 6,475,125,941 2,582,698,562 2,799,113,479 ㆍ매각예정자산 - 1,406,989,366 - [비유동자산] 3,486,870,206,480 3,057,587,028,807 2,838,916,582,682 ㆍ비유동금융자산 1,968,320,995,866 1,507,129,775,098 1,178,921,535,024 ㆍ장기매출채권및기타채권 3,836,297,908 639,067,147 743,963,687 ㆍ유형자산 1,253,724,035,719 1,234,405,624,615 1,218,920,525,514 ㆍ무형자산 1,263,382,019,647 1,288,620,525,514                                                                                                                                                         353,163,635,967 ㆍ기타자본구성요소 (212,564,793,120) (172,556,792,09

In [7]:
import re
import pandas as pd

def polish_sentence(text):
    # 1. 불필요한 반복 단어 제거
    text = re.sub(r'(같은 같은|등 등|및 및)', ' ', text)

    # 2. 너무 긴 문장은 쉼표 기준으로 나눔
    text = re.sub(r'([가-힣]), ([가-힣])', r'\1. \2', text)

    # 3. 조사 반복 수정
    text = re.sub(r'은 은|는 는|이 이|가 가|을 을|를 를', lambda m: m.group()[0], text)

    # 4. 문장 끝이 이상한 경우 마침표 붙임
    if not text.endswith(('다.', '다”', '요.', '임.', '임')):
        text = text.strip() + '다.'

    # 5. 구어체 → 문어체 간단 변환
    text = text.replace("하는 데", "하는 데에")
    text = text.replace("있는 것", "존재하는 것")
    text = text.replace("좀", "다소")
    text = text.replace("많이", "다수의")

    return text.strip()

# 경로의 정제본 파일을 다시 로드해서 문장 단위로 다듬기
input_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_요약결과전체.txt"
output_path = input_path.replace(".txt", "_문장정제.txt")

with open(input_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

# 문단 유지 + 문장별 다듬기
final = []
for block in raw_text.strip().split('\n\n'):
    if block.startswith("##"):
        final.append(block)
        continue
    sentences = re.split(r'(?<=[.?!])\s+', block)
    cleaned = [polish_sentence(s) for s in sentences if s]
    final.append(" ".join(cleaned))

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n\n".join(final).strip())

print("✅ 문장 정제 완료:", output_path)


✅ 문장 정제 완료: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_요약결과전체_문장정제.txt


In [8]:
print(final_output)



## 📊 재무 요약
## 📊 재무 요약 요약 연결 재무정보(K-IFRS) (단위 : 원) 제 27 기 제 26 기 제 25 기 구 분 (2024년 12월말) (2023년 12월말) (2022년 12월말) [유동자산] 1,213,770,948,090 1,391,068,809,421 1,328,836,088,014 ㆍ현금및현금성자산 244,499,636,973 166,186,546,142 101,060,567,371 ㆍ유동금융자산 932,676,085,313 1,187,637,044,467 1,198,553,807,206 ㆍ매출채권및기타채권 25,716,727,890 24,172,481,246 22,394,067,535 ㆍ재고자산 4,403,371,973 5,522,131,094 4,027,395,893 ㆍ당기법인세자산 - 3,560,918,544 1,136,530 ㆍ유동비금융자산 6,475,125,941 2,582,698,562 2,799,113,479 ㆍ매각예정자산 - 1,406,989,366 - [비유동자산] 3,486,870,206,480 3,057,587,028,807 2,838,916,582,682 ㆍ비유동금융자산 1,968,320,995,866 1,507,129,775,098 1,178,921,535,024 ㆍ장기매출채권및기타채권 3,836,297,908 639,067,147 743,963,687 ㆍ유형자산 1,253,724,035,719 1,234,405,624,615 1,218,920,525,514 ㆍ무형자산 1,263,382,019,647 1,288,620,525,514                                                                                                                                                         353,163,635,967 ㆍ기타자본구성요소 (212,564,793,120) (172,556,792,09

In [105]:
print(final_output)



## 📊 재무 요약
- (한국레저산업연구소 레저백서 2024 참고) 다) 회사의 경쟁상의 강점과 약점 등 (1) 강점 국내 최고 수준의 스키장, 골프장, 워터파크, 콘도 등을 보유한 휴양 레저시설로서 카지노와 호텔까지 한 곳에 모여있는 복합 리조트라는 점이 강점입니다.


## 🛡️ 리스크 및 부채
- 배당관련 예측가능성 제공에 관한 사항 1 정관상 배당절차 개선방안 이행 가부 구분 현황 및 계획 정관상 배당액 결정 기관 주주총회 가능 정관 제41조 제2항에 근거 정관상 배당기준일을 배당액 결정 이후로 정할  당사는 2024회계연도 결산배당부터 수 있는지 여부 배당기준일을 배당액 확정일정기주주총회일 이후의 날로 정하기로 함 배당절차 개선방안 이행 관련 향후 계획  2 배당액 확정일 및 배당기준일 지정 현황 배당액 배당 예측가능성 구분 결산월 배당여부 배당기준일 비고 확정일 제공여부 배당기준일 배당안 2024회계연도 202412 O 20250326 20250404 O 결정 및 공시일  결산배당 202537 2023회계연도 배당안 결정 및 공시일  202312 O 20240328 20231231 X 결산배당 202438 전자공시시스템 dartfssorkr Page 204 2022회계연도 배당안 결정 및 공시일  202212 O 20230329 20221231 X 결산배당 202339  배당액 확정일  정기주주총회일 다입니다.


## 🚀 사업 전략 및 성장
- 사업 전략 및 성장 연결대상 종속회사 현황상세 참조  연결대상회사의 변동내용 구 분 자회사 사 유 신규   연결   법인 청산 연결 주하이원엔터테인먼트  청산 종결 등기 완료2023919 제외  제27기2024회계연도부터 연결 제외    중소기업 등 해당 여부 중소기업 해당 여부 미해당 벤처기업 해당 여부 미해당 중견기업 해당 여부 미해당  회사의 주권상장또는 등록ㆍ지정 및 특례상장에 관한 사항 주권상장 주권상장 특례상장 유형 또는 등록ㆍ지정현황 또는 등록ㆍ지정일자 유가증권시장 상장 2003년 09월 0

In [106]:
# 🧩 필수 패키지 설치 (처음 1회만 실행)
!pip install transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 877.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 819.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [1]:
!pip install transformers sentence-transformers torch


## 전체 요약 파이프라인 (문장 정제 + 섹션 분류 = KoBART 요약)

In [5]:
import re
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# ✅ 경로 설정
input_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_요약결과전체.txt"
output_path = input_path.replace(".txt", "_딥러닝문장요약.txt")

# ✅ 1. 텍스트 로드 및 전처리
with open(input_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\n+', ' ', text)
sentences = re.split(r'(?<=[.?!])\s+', text)
print(f"✂️ 원본 문장 수: {len(sentences)}")

# ✅ 2. 잡음 문장 필터링
def is_valid_sentence(s):
    if len(s.strip()) < 10:
        return False
    digits = sum(c.isdigit() for c in s)
    if digits / max(len(s), 1) > 0.2:
        return False
    if len(re.findall(r'[가-힣a-zA-Z]', s)) < 5:
        return False
    return True

filtered_sentences = [s.strip() for s in sentences if is_valid_sentence(s)]
print(f"🧹 정제 후 문장 수: {len(filtered_sentences)}")

# ✅ 3. 문장 임베딩
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
sentence_embeddings = model.encode(filtered_sentences, convert_to_tensor=True)

# ✅ 4. 섹션 주제 정의 및 임베딩
topics = {
    "📊 재무 요약": "재무 성과 및 수익에 대한 요약",
    "🛡️ 리스크 및 부채": "부채 및 재무 리스크",
    "🚀 사업 전략 및 성장": "성장 전략과 비즈니스 계획",
    "🔬 기술 및 R&D": "기술 투자 및 연구개발",
    "🌐 글로벌 진출": "해외 시장 및 글로벌 전략",
    "🏢 조직 및 인력": "조직 구성 및 인력 운영"
}
topic_sentences = list(topics.values())
topic_embeddings = model.encode(topic_sentences, convert_to_tensor=True)

# ✅ 5. 요약 파이프라인 준비 (KoBART)
summarizer = pipeline("summarization", model="digit82/kobart-summarization", tokenizer="digit82/kobart-summarization", device=0 if torch.cuda.is_available() else -1)

# ✅ 6. 섹션별 유사 문장 추출 후 요약
summary_result = {}
for i, topic in enumerate(topics.keys()):
    cosine_scores = util.pytorch_cos_sim(topic_embeddings[i], sentence_embeddings)[0]
    k = min(5, len(cosine_scores))  # 유사도 상위 문장 개수
    top_k = torch.topk(cosine_scores, k=k)
    top_sentences = [filtered_sentences[idx] for idx in top_k.indices]
    joined_text = " ".join(top_sentences)

    summary = summarizer(joined_text, max_length=60, min_length=20, do_sample=False)[0]["summary_text"]
    summary_result[topic] = summary.strip()

# ✅ 7. 결과 출력 및 저장
final_output = ""
for sec, summary in summary_result.items():
    final_output += f"\n\n## {sec}\n- {summary}\n"

with open(output_path, "w", encoding="utf-8") as f:
    f.write(final_output.strip())

print("✅ 최종 요약 완료:", output_path)


✂️ 원본 문장 수: 21
🧹 정제 후 문장 수: 9


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (h

✅ 최종 요약 완료: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries/250723_kobert_summarization_요약결과전체_딥러닝문장요약.txt


In [8]:
print(final_output)



## 📊 재무 요약
- 국내 최고 수준의 스키장, 골프장, 워터파크, 콘도 등을 보유한 휴양 레저시설로서 카지노와 호텔까지 한 곳에 모여있는 복합 리조트라는 점이 강점이고 국내 최고 수준의 스키장, 골프장, 워터파크, 콘도 등을 보유한 휴양 레저시설로서 카지노와 호텔까지 한 곳에 모여있는 복합 리조트라는 점이 강점이라고 한다.


## 🛡️ 리스크 및 부채
- 당사는 2024회계연도 결산배당부터 수 있는지 여부 배당기준일을 배당액 확정일(정기주주총회일) 이후의 날로 정하기로 함 배당절차 개선방안 이행에 관한 사항 1) 정관상 배당절차 개선방안 이행 가부 구분 현황 및 계획 정관상 배당액 결정 이후로 정할 ※ 당사는 2024회계연도 결산배당부터 수 있는지 여부 배당기준일을 배당액 확정일(정기주주총회일) 이후의 날로 정하기로 함 배당절차 개선방안 이행 관련 향후 계획 - 2) 배당액 확정일 및 배당기준일 지정 현황 배당액 배당 예측가능성 구분 결산월 배당여부 배당기준일 지정 현황 배당액 배당 예측가능성 구분 결산월 배당여부 배당기준일 제공여부 배당기준일 제공여부 2024회계연도 2024.12 O 2025.03.26 2025.04.04 O 결정 및 공시일 : 결산배당 2025.32 O 2025.32 O 2024.03.28 X 결산배당 2023.32.31 X 결산배당 2023.32.31 X 결산배당 2023.32.31 X 결산배당 2023.12.31 X 결산배당 2024.32


## 🚀 사업 전략 및 성장
- 정관상 배당절차 개선방안 이행에 관한 사항 1) 정관상 배당절차 개선방안 이행 가부 구분 현황 및 계획 정관상 배당액 결정 이후로 정할 ※ 당사는 2024회계연도 결산배당부터 수 있는지 여부 배당기준일을 배당액 확정일(정기주주총회일) 이후의 날로 정하기로 함 배당절차 개선방안 이행 관련 향후 계획 - 2) 배당액 확정일 및 배당기준일 지정 현황 배당액 배당 예측가능성 구분 결산월 배당여부 배당기준일 비고 확정일 제공여부 배당기준일 제공여부 배당기준일, 배당

# fineBART 한국어 모델

---

